In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
import json

# Import API key
from config import gkey

ModuleNotFoundError: No module named 'config'

In [ ]:
#Load the csv exported in Part I to a DataFrame
world_weather_df = pd.read_csv("output_data/city_weather.csv")
world_weather_df

In [ ]:
#Humidity Heatmap
#Configure gmaps.
gmaps.configure(api_key = gkey)
#Use the Lat and Lng as locations and Humidity as the weight.
locations = world_weather_df[["Lat","Lng"]]
weight = world_weather_df["Humidity"]
#Add Heatmap layer to map.
gmaps.configure(api_key=gkey)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights = weight)
fig.add_layer(heatmap_layer)
fig

In [5]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
ideal_weather = world_weather_df[world_weather_df['Max Temp']>=70]
ideal_weather = ideal_weather[ideal_weather['Max Temp']<80]
ideal_weather = ideal_weather[ideal_weather['Wind Speed']<10]
ideal_weather = ideal_weather[ideal_weather['Cloudiness']==0]
ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,pisco,-13.7000,-76.2167,75.00,73,0,8.05,PE,1611159747
40,40,salalah,17.0151,54.0924,78.80,36,0,4.61,OM,1611159749
76,76,hermanus,-34.4187,19.2345,77.00,59,0,7.00,ZA,1611159754
99,99,burla,21.5000,83.8667,71.74,49,0,6.13,IN,1611159757
142,142,najran,17.4924,44.1277,71.60,15,0,4.61,SA,1611159483
210,210,guhagar,17.4842,73.1929,76.73,63,0,6.67,IN,1611159771
507,507,gawler,-34.6000,138.7333,70.00,24,0,1.01,AU,1611159809
524,524,acajutla,13.5928,-89.8275,78.80,53,0,4.61,SV,1611159811
532,532,bilma,18.6853,12.9164,73.92,19,0,5.97,NE,1611159813


In [6]:
ideal_weather["Hotel"]=""
ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
25,25,pisco,-13.7000,-76.2167,75.00,73,0,8.05,PE,1611159747,
40,40,salalah,17.0151,54.0924,78.80,36,0,4.61,OM,1611159749,
76,76,hermanus,-34.4187,19.2345,77.00,59,0,7.00,ZA,1611159754,
99,99,burla,21.5000,83.8667,71.74,49,0,6.13,IN,1611159757,
142,142,najran,17.4924,44.1277,71.60,15,0,4.61,SA,1611159483,
210,210,guhagar,17.4842,73.1929,76.73,63,0,6.67,IN,1611159771,
507,507,gawler,-34.6000,138.7333,70.00,24,0,1.01,AU,1611159809,
524,524,acajutla,13.5928,-89.8275,78.80,53,0,4.61,SV,1611159811,
532,532,bilma,18.6853,12.9164,73.92,19,0,5.97,NE,1611159813,


In [7]:
#Finding the nearest hotel to each city
params = {
    "rankby":"distance",
    "types": "lodging",
    "key": gkey}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_weather.iterrows():
    #get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    #change location each iteration while leaving original params in place
    params['location'] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    #make request to the url
    hotels_cities = requests.get(base_url, params=params)
    hotels_cities = hotels_cities.json()
    
    # Fill in the hotel column 
    try:
        ideal_weather.loc[index, "Hotel"] = hotels_cities["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [8]:
#Verify that the closest hotel is showing
ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
25,25,pisco,-13.7000,-76.2167,75.00,73,0,8.05,PE,1611159747,Marcos Saldarriaga Salguero
40,40,salalah,17.0151,54.0924,78.80,36,0,4.61,OM,1611159749,Muscat International Hotel plaza Salalah
76,76,hermanus,-34.4187,19.2345,77.00,59,0,7.00,ZA,1611159754,Aloe House Guest Lodge
99,99,burla,21.5000,83.8667,71.74,49,0,6.13,IN,1611159757,Nanda’s Manor
142,142,najran,17.4924,44.1277,71.60,15,0,4.61,SA,1611159483,محمد هادي ال فطيح
210,210,guhagar,17.4842,73.1929,76.73,63,0,6.67,IN,1611159771,Prestige Deluxe
507,507,gawler,-34.6000,138.7333,70.00,24,0,1.01,AU,1611159809,Getaway Cottage
524,524,acajutla,13.5928,-89.8275,78.80,53,0,4.61,SV,1611159811,asi es
532,532,bilma,18.6853,12.9164,73.92,19,0,5.97,NE,1611159813,Expeditions Ténére voyages


In [9]:
# Convert hotels as a list
hotel_names = ideal_weather["Hotel"].tolist()

In [10]:
#Create a map using city coordinates to set markers
marker_locations = ideal_weather[["Lat","Lng"]]

#Create a marker using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
                            info_box_content = [f" Hotel: {hotel}" for hotel in hotel_names])
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

In [11]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heatmap_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))